In [1]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tslearn.utils import to_time_series_dataset
import biosppy.signals.ecg as ecg
import matplotlib.pyplot as plt
import neurokit2 as nk
import heartpy as hp
from statistics import mean
import functions
from multiprocessing import Pool
from datetime import time
from pandarallel import pandarallel
import functions
import scipy
import math
import pywt
import operator
from functions import *
import itertools

In [2]:
folder = ""
raw = folder + "raw_data/"
x_train = pd.read_csv('x_train_preprocess.csv')
x_test = pd.read_csv('x_test_preprocess.csv')
y_train = pd.read_csv(raw + 'y_train.csv', index_col=['id'])
x_train = x_train.fillna(0)
x_test = x_test.fillna(0)

In [3]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5117 entries, 0 to 5116
Columns: 1009 entries, Unnamed: 0 to nk_feat_387
dtypes: float64(1008), int64(1)
memory usage: 39.4 MB


In [4]:
selector = SelectKBest(score_func=f_classif, k=150)
selector.fit(np.array(x_train), np.array(y_train).squeeze())
cols = selector.get_support(indices=True)

/home/francescog/Scaricati/AML_Pr_2/venv/lib/python3.8/site-packages/sklearn/feature_selection/_univariate_selection.py:110: UserWarning: Features [72] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)


In [5]:
#scale data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [6]:
from sklearn import svm
svm_model = svm.SVC(random_state=0, probability=True)

In [7]:
#cross validation score
scores = cross_val_score(svm_model, x_train, np.ravel(y_train), cv=10, scoring='f1_micro', n_jobs=-1, verbose=1)
print(scores.mean())
print(scores.std())
#0.6738265961350293 / 0.013598950995017234 StandardScaler
#0.6011325067270058 / 0.004216629008159178 MinMaxScaler
#0.6820350721624268 / 0.010050291146522606 new preprocessing
#0.6947391756360078 / 0.011488590054798726 new new preprocessing
#0.7396824547455968 / 0.01680257443527899 1000 feat

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


0.7396824547455968
0.01680257443527899


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 14.7min finished


In [8]:
import pickle
filename = 'models/svm_model.sav'
pickle.dump(svm_model, open(filename, 'wb'))


In [9]:
svm_model.fit(x_train, np.ravel(y_train))
y_test = svm_model.predict(x_test)
data_y_test = {
    'id': [i for i in range(len(y_test))],
    'y': y_test
}
df_y_test = pd.DataFrame(data_y_test)
df_y_test.to_csv('outputs/output_svm.csv', index=False)